# Keras Digit Recogniser

In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Flatten
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

Using TensorFlow backend.
/Users/james.smith1@ibm.com/anaconda2/envs/untitled2/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
df_data_1 = pd.read_csv('train.csv')
df_data_2 = pd.read_csv('test.csv')
train = df_data_1
test = df_data_2.values.astype('float32')

In [3]:
# preview data
train.head()
#test.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
labels = train.label.values.astype('int32')
train_img = train.loc[:,'pixel0':].values.astype('float32')
#labels
train_img.shape, test.shape
#type(labels)

((42000, 784), (28000, 784))

In [5]:
# convert labels to a categorical data type, as expected by keras model
labels = to_categorical(labels)
classes = labels.shape[1]
labels

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [6]:
# fix random seed for reproducibility
seed = 43
np.random.seed(seed)

## Multilayer Perceptron

Now we will build the neural network. We use a Keras Sequential object to build a simple feedforward network (multilayer perceptron, or MLP), with Relu input and hidden layer activations, and a softmax output layer with 10 classes. model.add() will be used to add layers.

In [7]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=784))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
model_fit = model.fit(train_img, labels, epochs=100, batch_size=64, validation_split=0.1, callbacks=[early_stopping])

Train on 37800 samples, validate on 4200 samples
Epoch 1/100
37800/37800 [==============================] - 3s 78us/step - loss: 0.7105 - acc: 0.7716 - val_loss: 0.3533 - val_acc: 0.9017
Epoch 2/100
37800/37800 [==============================] - 2s 54us/step - loss: 0.2843 - acc: 0.9197 - val_loss: 0.2499 - val_acc: 0.9324
Epoch 3/100
37800/37800 [==============================] - 2s 54us/step - loss: 0.2198 - acc: 0.9400 - val_loss: 0.2225 - val_acc: 0.9405
Epoch 4/100
37800/37800 [==============================] - 2s 54us/step - loss: 0.1849 - acc: 0.9486 - val_loss: 0.2100 - val_acc: 0.9457
Epoch 5/100
37800/37800 [==============================] - 2s 54us/step - loss: 0.1626 - acc: 0.9540 - val_loss: 0.2116 - val_acc: 0.9421
Epoch 6/100
37800/37800 [==============================] - 2s 53us/step - loss: 0.1535 - acc: 0.9575 - val_loss: 0.2043 - val_acc: 0.9448
Epoch 7/100
37800/37800 [==============================] - 2s 55us/step - loss: 0.1390 - acc: 0.9623 - val_loss: 0.1878 - v

In [10]:
pred = model.predict_classes(test)
pred, len(pred)

(array([2, 0, 9, ..., 3, 9, 2]), 28000)

In [11]:
result = pd.DataFrame({'ImageId':list(range(1,test.shape[0]+1)),
                       'Label':pred})
result.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,4
4,5,3


In [12]:
result.to_csv('mlp_output.csv', index=False, header=True)

## Convolutional Neural Network

The MLP gives good accuracy, but is a generic model, not specialised for image data. Convolutional Neural Networks (CNNs) are specifically designed for processing grid-shaped data such as images, and should, theoretically, have much greater statistical efficiency when applied to this task, achieving better results.

In [13]:
cnn = Sequential()
cnn.add(Conv2D(4, (3,3), padding='same', input_shape=(28,28,1)))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(8, (3,3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(Conv2D(8, (3,3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Flatten())
cnn.add(Dense(32, activation='relu'))
cnn.add(Dense(10, activation='softmax'))
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

train_img_cnn = train_img.reshape(train_img.shape[0],28,28,1)

cnn_fit = cnn.fit(train_img_cnn, labels, epochs=100, batch_size=64, validation_split=0.1, callbacks=[early_stopping])

Train on 37800 samples, validate on 4200 samples
Epoch 1/100
37800/37800 [==============================] - 14s 378us/step - loss: 2.6536 - acc: 0.4114 - val_loss: 0.5652 - val_acc: 0.8243
Epoch 2/100
37800/37800 [==============================] - 14s 362us/step - loss: 0.5519 - acc: 0.8187 - val_loss: 0.2347 - val_acc: 0.9283s: 0.5927 - acc: 0.805 - ETA: 2s - loss: 0.5918 - acc: - ETA: 2s - loss: 0.5828 - acc: 0.8 - ETA: 2s - loss: 0.5802 - acc: 0.80 - ETA: 2s - loss - ETA: 0s - loss: 0.5583 - a
Epoch 3/100
37800/37800 [==============================] - 14s 360us/step - loss: 0.3246 - acc: 0.8953 - val_loss: 0.1587 - val_acc: 0.9490
Epoch 4/100
37800/37800 [==============================] - 13s 351us/step - loss: 0.2380 - acc: 0.9242 - val_loss: 0.1163 - val_acc: 0.9614
Epoch 5/100
37800/37800 [==============================] - 14s 364us/step - loss: 0.1993 - acc: 0.9360 - val_loss: 0.0963 - val_acc: 0.9690
Epoch 6/100
37800/37800 [==============================] - 13s 355us/step - lo

In [15]:
pred = cnn.predict_classes(test.reshape(test.shape[0],28,28,1))
result = pd.DataFrame({'ImageId':list(range(1,test.shape[0]+1)),
                       'Label':pred})
result.to_csv('cnn_output.csv', index=False, header=True)